<a href="https://colab.research.google.com/github/yalopez84/Interpretable_method/blob/master/Fine_tuning_sbert_mnr_with_TSDAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.8 MB/s 
     |████████████████████████████████| 4.4 MB 26.3 MB/s 
     |████████████████████████████████| 1.2 MB 19.2 MB/s 
     |████████████████████████████████| 101 kB 5.6 MB/s 
     |████████████████████████████████| 596 kB 20.9 MB/s 
     |████████████████████████████████| 6.6 MB 43.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=fbbfa9165819d7ad3ce40260ad288794919c31d330e940d2fcc1452f9b818f07
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-w

In [2]:
from sentence_transformers import SentenceTransformer
import datasets
import os
import csv
import sys
import random
from tqdm import tqdm, trange
import logging
import pdb
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbs
from torch.utils.tensorboard import SummaryWriter
writter = SummaryWriter(log_dir='tensorboard_embedding_custom2')
logger = logging.getLogger(__name__)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.system("export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:20")

0

# Preprocessing-Start

In [5]:
class InputExample(object):

    def __init__(self, guid=None, text_a=None, text_b=None, text_c=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.text_c = text_c
        self.label = label

In [6]:
class Triple(object):
    def __init__(self, guid, subject , predicate , obj, label):
        self.guid=guid
        self.subject=subject
        self.predicate=predicate
        self.obj=obj
        self.label=label

In [7]:
class ExampleTruncado(object):

    def __init__(self, text, label=None):
        self.text = text
        self.label = label

In [8]:
class DataProcessor(object):
  
    def get_train_examples(self, data_dir):
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        raise NotImplementedError()

    def get_labels(self, data_dir):
        raise NotImplementedError()

    def get_train_with_negatives(self,data_dir):
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            #pdb.set_trace()            
            for line in reader:
                lines.append(line)
            return lines
            
    @classmethod
    def _read_train_with_negatives(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter=" ", quotechar=quotechar)
            lines = []
            for line in reader:         
                lines.append(line)
            return lines

In [9]:
class KGProcessor(DataProcessor):
    def __init__(self):
        self.labels = set()
    
    def get_train_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train", data_dir)
        
    def get_dev_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev", data_dir)

    def get_test_examples(self, data_dir):
        return self._create_examples(
          self._read_tsv(os.path.join(data_dir, "test_reduced.tsv")), "test", data_dir)
        
    def get_train_with_negatives(self,data_dir):
        return self._create_examples_with_negatives(
          self._read_train_with_negatives(os.path.join(data_dir, "train_reduced_with_negatives.txt")), "embeddings", data_dir)
        
    def get_train_without_negatives(self,data_dir):
        return self._create_examples_without_negatives(
          self._read_tsv(os.path.join(data_dir, "train_reduced.tsv")), "embeddings", data_dir)

    def get_relations(self, data_dir):
        with open(os.path.join(data_dir, "relations.txt"), 'r') as f:
            lines = f.readlines()
            relations = []
            for line in lines:
                relations.append(line.strip())
        return relations

    def get_labels(self, data_dir):
        return ["0", "1"]

    def get_entities(self, data_dir):
        with open(os.path.join(data_dir, "entities.txt"), 'r') as f:
            lines = f.readlines()
            entities = []
            for line in lines:
                entities.append(line.strip())
        return entities

    def get_prediction_test_triples(self, data_dir):
        with open(os.path.join(data_dir, "test_results.txt"), 'r') as f:
            lines = f.readlines()
            predictions = []
            for line in lines:               
                predictions.append(line.split()[2])
        return predictions

    def get_train_triples(self, data_dir):
        return self._read_train_with_negatives(os.path.join(data_dir, "train_reduced_with_negatives.txt"))    

    def get_dev_triples(self, data_dir):
        return self._read_tsv(os.path.join(data_dir, "dev.tsv"))

    def get_test_triples(self, data_dir):
        return self._read_tsv(os.path.join(data_dir, "test_reduced.tsv"))

    def _create_examples(self, lines, set_type, data_dir):
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in tqdm(ent_lines):
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    ent2text[temp[0]] = temp[1]

        entities = list(ent2text.keys())

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]      

        lines_str_set = set(['\t'.join(line) for line in lines])
        examples = []
        for (i, line) in enumerate(lines):
            auxiliar=["loke_joseph_of_austria_palatine_of_hungary","united_srena_gale", "archduke_joseph_of_austria_palatine_of_hungary"]
            flag_aux=False
            if line[0] or line[2] not in auxiliar:
                head_ent_text = ent2text[line[0]]
                tail_ent_text = ent2text[line[2]]
                relation_text = rel2text[line[1]]
                flag_aux=True
            else:
                head_ent_text = line[0]
                tail_ent_text = line[2]
                relation_text = line[1]
                flag_aux=True

            if set_type == "dev" or set_type == "test" and flag_aux:
                triple_label = line[3]
                if triple_label == "1":
                    label = "1"
                else:
                    label = "0"

                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 
                self.labels.add(label)
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label=label))
                
            elif set_type == "train" and flag_aux:
                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label="1"))

                rnd = random.random()
                guid = "%s-%s" % (set_type + "_corrupt", i)
                if rnd <= 0.5:
                    # corrupting head
                    tmp_head = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[0])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_head = random.choice(tmp_ent_list)
                        tmp_triple_str = tmp_head + '\t' + line[1] + '\t' + line[2]
                        if tmp_triple_str not in lines_str_set:
                            break                    
                    tmp_head_text = ent2text[tmp_head]
                    examples.append(
                        InputExample(guid=guid, text_a=tmp_head_text, text_b=text_b, text_c = text_c, label="0"))       
                else:
                    # corrupting tail
                    tmp_tail = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[2])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_tail = random.choice(tmp_ent_list)
                        tmp_triple_str = line[0] + '\t' + line[1] + '\t' + tmp_tail
                        if tmp_triple_str not in lines_str_set:
                            break
                    tmp_tail_text = ent2text[tmp_tail]
                    examples.append(
                        InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = tmp_tail_text, label="0"))      
                                                            
        return examples

    def _create_examples_without_negatives(self, lines, set_type, data_dir):
        
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in tqdm(ent_lines):
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    ent2text[temp[0]] = temp[1]

        entities = list(ent2text.keys())

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]      

        examples = []
        for (i, line) in enumerate(lines):
            auxiliar=["loke_joseph_of_austria_palatine_of_hungary","united_srena_gale", "archduke_joseph_of_austria_palatine_of_hungary"]
            flag_aux=False            
            if len(line)==3:
                if line[0] or line[2] not in auxiliar:               
                    head_ent_text = ent2text[line[0]]                
                    relation_text = rel2text[line[1]]
                    tail_ent_text = ent2text[line[2]]
                    examples.append(
                        InputExample(text_a=head_ent_text, text_b=relation_text, text_c = tail_ent_text))     
                                                           
        return examples

    def _create_examples_with_negatives(self, lines, set_type, data_dir):
        
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in tqdm(ent_lines):
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    ent2text[temp[0]] = temp[1]

        entities = list(ent2text.keys())

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]      

        examples = []
        for (i, line) in enumerate(lines):
            auxiliar=["loke_joseph_of_austria_palatine_of_hungary","united_srena_gale", "archduke_joseph_of_austria_palatine_of_hungary"]
            flag_aux=False
            if len(line)==5:
                if line[1] or line[3] not in auxiliar:               
                    guid=line[0]
                    head_ent_text = ent2text[line[1]]                
                    relation_text = rel2text[line[2]]
                    tail_ent_text = ent2text[line[3]]
                    triple_label = line[4]
                    examples.append(
                        InputExample(guid=guid, text_a=head_ent_text, text_b=relation_text, text_c = tail_ent_text, label=triple_label))     
                                                           
        return examples

In [10]:
def convert_examples_to_examplesTruncados(examples, max_seq_length, label_list=None, tokenizer=None, print_info = True):

    examples_truncados = []

    for (ex_index, example) in tqdm(enumerate(examples)):
        
        tokens_a = example.text_a.split()
        tokens_b = None
        tokens_c = None
       
        if example.text_b and example.text_c:
            tokens_b = example.text_b.split()
            tokens_c = example.text_c.split()
           
            _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_seq_length)
        else:
            if len(tokens_a) > max_seq_length:
                tokens_a = tokens_a[:(max_seq_length)]
        tokens =tokens_a.copy()

        if tokens_b:
            tokens.extend(tokens_b)           
        if tokens_c:
            tokens.extend(tokens_c)        

        padding = ["0"] * (max_seq_length - len(tokens))
        tokens.extend(padding)
        assert len(tokens) == max_seq_length

        examples_truncados.append(
                ExampleTruncado(text=tokens,label=example.label))
    return examples_truncados

In [11]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        sujeto=tokens_a.split()
        relation=tokens_b.split()
        total_length = len(sujeto) + len(relation)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [12]:
def _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b) + len(tokens_c)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b) and len(tokens_a) > len(tokens_c):
            tokens_a.pop()
        elif len(tokens_b) > len(tokens_a) and len(tokens_b) > len(tokens_c):
            tokens_b.pop()
        elif len(tokens_c) > len(tokens_a) and len(tokens_c) > len(tokens_b):
            tokens_c.pop()
        else:
            tokens_c.pop()

# Preprocessing-End

In [13]:
pathFile="/content/drive/MyDrive/Almacen_phd/FB13/"
modelName="/content/drive/MyDrive/Almacen_phd/FB13/sbert_test_mnr2"
os.chdir(pathFile)
flag_sbert_test_mnr2=True 
flag_TSDAE=True         
flag_save_model=True    

In [14]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt = '%m/%d/%Y %H:%M:%S',
    level = logging.INFO)

processor = KGProcessor()  
label_list = processor.get_labels(pathFile)

In [15]:
_examples_training = processor.get_train_without_negatives(pathFile) 
_examples_training_truncates=convert_examples_to_examplesTruncados(_examples_training,128)
X_train=[]
for sample in _examples_training_truncates:
    X_train.append(" ".join(sample.text))

100%|██████████| 75042/75042 [00:00<00:00, 334924.83it/s]
11082it [00:04, 2519.14it/s]


In [16]:
from sentence_transformers.datasets import DenoisingAutoEncoderDataset
from torch.utils.data import DataLoader

# dataset class with noise functionality built-in
train_data = DenoisingAutoEncoderDataset(X_train)

# we use a dataloader as usual
loader = DataLoader(train_data, batch_size=8, shuffle=True, drop_last=True)

In [17]:
from sentence_transformers import SentenceTransformer, models

bert = models.Transformer('bert-base-uncased')
pooling = models.Pooling(bert.get_word_embedding_dimension(), 'cls')

model = SentenceTransformer(modules=[bert, pooling])
model

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

07/21/2022 15:47:07 - INFO - sentence_transformers.SentenceTransformer -   Use pytorch device: cuda


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [18]:
from sentence_transformers.losses import DenoisingAutoEncoderLoss

loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.10.crossattention.self.query.weight', 'bert.encoder.layer.6.crossattention.self.query.weight', 'bert.encoder.layer.3.crossattention.output.LayerNorm.weight', 'bert.

In [19]:
!pip install nltk
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
if flag_sbert_test_mnr2:
    model = SentenceTransformer(modelName)

if flag_TSDAE:
    model.fit(
        train_objectives=[(loader, loss)],
        epochs=1,
        weight_decay=0,
        scheduler='constantlr',
        optimizer_params={'lr': 3e-5},
        show_progress_bar=True
    )

07/21/2022 15:47:15 - INFO - sentence_transformers.SentenceTransformer -   Load pretrained SentenceTransformer: /content/drive/MyDrive/Almacen_phd/FB13/sbert_test_mnr2
07/21/2022 15:47:24 - INFO - sentence_transformers.SentenceTransformer -   Use pytorch device: cuda


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1385 [00:00<?, ?it/s]

In [21]:
if flag_save_model:
    model.save('output/tsdae-bert-base-uncased')

07/21/2022 16:01:24 - INFO - sentence_transformers.SentenceTransformer -   Save model to output/tsdae-bert-base-uncased


In [ ]:
sts = datasets.load_dataset('glue', 'stsb', split='validation')
sts

In [23]:
sts = sts.map(lambda x: {'label': x['label'] / 5.0})

07/21/2022 16:01:41 - WARNING - datasets.fingerprint -   Parameter 'function'=<function <lambda> at 0x7fbf40a144d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1500 [00:00<?, ?ex/s]

In [24]:
from sentence_transformers import InputExample
samples = []
for sample in sts:
    # reformat to use InputExample
    samples.append(InputExample(
        texts=[sample['sentence1'], sample['sentence2']],
        label=sample['label']
    ))

In [25]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    samples, write_csv=False
)

In [26]:
evaluator(model)

07/21/2022 16:01:42 - INFO - sentence_transformers.evaluation.EmbeddingSimilarityEvaluator -   EmbeddingSimilarityEvaluator: Evaluating the model on  dataset:
07/21/2022 16:01:47 - INFO - sentence_transformers.evaluation.EmbeddingSimilarityEvaluator -   Cosine-Similarity :	Pearson: 0.8376	Spearman: 0.8389
07/21/2022 16:01:47 - INFO - sentence_transformers.evaluation.EmbeddingSimilarityEvaluator -   Manhattan-Distance:	Pearson: 0.8342	Spearman: 0.8321
07/21/2022 16:01:47 - INFO - sentence_transformers.evaluation.EmbeddingSimilarityEvaluator -   Euclidean-Distance:	Pearson: 0.8345	Spearman: 0.8323
07/21/2022 16:01:47 - INFO - sentence_transformers.evaluation.EmbeddingSimilarityEvaluator -   Dot-Product-Similarity:	Pearson: 0.7800	Spearman: 0.7692


0.8388895896028158